In [1]:
pip install tensorflow==2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 27.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  At

In [ ]:
# Import Keras modules and its important APIs
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, Flatten, Dense, Add
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import GlorotUniform, Constant

# Identity Block
def identity_block(X, f, filters, training=True, initializer=GlorotUniform):
    F1, F2, F3 = filters
    X_shortcut = X

    # First component of main path
    if initializer == Constant:
        X = Conv2D(filters=F1, kernel_size=1, strides=(1, 1), padding='valid', kernel_initializer=initializer())(X)
    else:
        X = Conv2D(filters=F1, kernel_size=1, strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=initializer())(X) if initializer == Constant else Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer())(X) if initializer == Constant else Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)

    # Add shortcut and pass through RELU
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

# Convolutional Block
def convolutional_block(X, f, filters, s=2, training=True, initializer=GlorotUniform):
    F1, F2, F3 = filters
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=1, strides=(s, s), padding='valid', kernel_initializer=initializer() if initializer == Constant else initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=initializer() if initializer == Constant else initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer() if initializer == Constant else initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)

    # Shortcut path
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_initializer=initializer() if initializer == Constant else initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut, training=training)

    # Final step
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

# ResNet50 Model
def ResNet50(input_shape=(64, 64, 3), classes=10):
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer=GlorotUniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # Average Pooling and Output layer
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer=GlorotUniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X)

    return model

# Instantiate the model and print summary
model = ResNet50(input_shape=(64, 64, 3), classes=10)
print(model.summary())


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#print("Unique labels in Y_train_orig:", np.unique(Y_train_orig))
#print("Unique labels in Y_test_orig:", np.unique(Y_test_orig))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load the CIFAR-10 dataset
(X_train_orig, Y_train_orig), (X_test_orig, Y_test_orig) = cifar10.load_data()

# Normalize image vectors
X_train = X_train_orig / 255.0
X_test = X_test_orig / 255.0

# Resize images to 64x64
X_train = tf.image.resize(X_train, [64, 64])
X_test = tf.image.resize(X_test, [64, 64])

# Convert training and test labels to one-hot matrices
Y_train = to_categorical(Y_train_orig, num_classes=10)
Y_test = to_categorical(Y_test_orig, num_classes=10)

# Print shapes
print("number of training examples =", X_train.shape[0])
print("number of test examples =", X_test.shape[0])
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)

# Fit the model
model.fit(X_train, Y_train, epochs=10, batch_size=32)


170498071/170498071 [==============================] - 6s 0us/step
number of training examples = 50000
number of test examples = 10000
X_train shape: (50000, 64, 64, 3)
Y_train shape: (50000, 10)
X_test shape: (10000, 64, 64, 3)
Y_test shape: (10000, 10)
Epoch 1/10
  75/1563 [>.............................] - ETA: 2:58:38 - loss: 3.4352 - accuracy: 0.1762

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Define class labels for CIFAR-10
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Get the path of the image to be predicted
input_image_path = input('Path of the image to be predicted: ')

# Load the image
input_image = cv2.imread(input_image_path)

# Display the image
cv2_imshow(input_image)

# Resize the image to (32, 32) for CIFAR-10
input_image_resized = cv2.resize(input_image, (32, 32))

# Normalize the image
input_image_scaled = input_image_resized / 255.0

# Reshape the image to fit the model input
input_image_reshaped = np.reshape(input_image_scaled, [1, 32, 32, 3])

# Make a prediction
input_prediction = model.predict(input_image_reshaped)

# Print the prediction
print("Prediction vector:", input_prediction)

# Get the predicted class label
input_pred_label = np.argmax(input_prediction)

# Print the predicted class name
print("Predicted class:", class_labels[input_pred_label])
